# Neural Network

Reference: https://www.kaggle.com/c/GiveMeSomeCredit, https://www.kaggle.com/code

![](https://pbs.twimg.com/media/CsuoZJwWgAAdaD6.jpg)

Credit (Image and Good Read): https://www.asimovinstitute.org/neural-network-zoo/

## SciKit-Learn

In order to follow along with this tutorial, you'll need to have the latest version of SciKit Learn installed! It is easily installable either through pip or conda, but you can reference the [official installation documentation](http://scikit-learn.org/stable/install.html) for complete details on this.


## Data

Improve on the state of the art in credit scoring by predicting the probability that somebody will experience financial distress in the next two years.

In [27]:
!pip install --upgrade scikit-learn==1.0.2
!pip install --upgrade numpy==1.21.5

In [28]:
import pandas as pd
data = pd.read_csv('https://github.com/davidjohnnn/all_datasets/raw/master/bay/cs-training.csv').drop('Unnamed: 0', axis = 1)

![](https://thumbor.forbes.com/thumbor/fit-in/900x510/https://www.forbes.com/advisor/wp-content/uploads/2020/05/banks-credit-chalkboard_getty.jpg)

Let's check out the data:

In [29]:
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [30]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
SeriousDlqin2yrs,150000.0,0.066840,0.249746,0.0,0.000000,0.000000,0.000000,1.0
RevolvingUtilizationOfUnsecuredLines,150000.0,6.048438,249.755371,0.0,0.029867,0.154181,0.559046,50708.0
age,150000.0,52.295207,14.771866,0.0,41.000000,52.000000,63.000000,109.0
NumberOfTime30-59DaysPastDueNotWorse,150000.0,0.421033,4.192781,0.0,0.000000,0.000000,0.000000,98.0
DebtRatio,150000.0,353.005076,2037.818523,0.0,0.175074,0.366508,0.868254,329664.0
MonthlyIncome,120269.0,6670.221237,14384.674215,0.0,3400.000000,5400.000000,8249.000000,3008750.0
NumberOfOpenCreditLinesAndLoans,150000.0,8.452760,5.145951,0.0,5.000000,8.000000,11.000000,58.0
NumberOfTimes90DaysLate,150000.0,0.265973,4.169304,0.0,0.000000,0.000000,0.000000,98.0
NumberRealEstateLoansOrLines,150000.0,1.018240,1.129771,0.0,0.000000,1.000000,2.000000,54.0
NumberOfTime60-89DaysPastDueNotWorse,150000.0,0.240387,4.155179,0.0,0.000000,0.000000,0.000000,98.0


In [31]:
data.shape

(150000, 11)

Let's set up our Data and our Labels:

In [32]:
data.dropna(axis=0, inplace=True)

X = data.drop('SeriousDlqin2yrs',axis=1)
y = data['SeriousDlqin2yrs']

### Train Test Split

Let's split our data into training and testing sets, this is done easily with SciKit Learn's train_test_split function from model_selection:

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=101)

## Data Preprocessing

The neural network may have difficulty converging before the maximum number of iterations allowed if the data is not normalized. Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. Note that you must apply the same scaling to the test set for meaningful results.  There are a lot of different methods for normalization of data, we will use the built-in StandardScaler for standardization.

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
scaler = StandardScaler()

In [37]:
# Fit only to the training data
scaler.fit(X_train)

StandardScaler()

In [38]:
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Training the model

Now it is time to train our model. SciKit Learn makes this incredibly easy, by using estimator objects. In this case we will import our estimator (the Multi-Layer Perceptron Classifier model) from the neural_network library of SciKit-Learn!

In [39]:
from sklearn.neural_network import MLPClassifier

In [40]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100), learning_rate_init=0.001, max_iter=500, early_stopping=True, random_state=101)

In [41]:
mlp.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(100, 100), max_iter=500,
              random_state=101)

You can see the output that shows the default values of the other parameters in the model. I encourage you to play around with them and discover what effects they have on your model!

## Predictions and Evaluation

Now that we have a model it is time to use it to get predictions! We can do this simply with the predict() method off of our fitted model:

In [42]:
predictions = mlp.predict(X_test)

In [43]:
print(X.columns) # feature names
print(y.unique().tolist()) # class names

Index(['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')
[1, 0]


Now we can use SciKit-Learn's built in metrics such as a classification report and confusion matrix to evaluate how well our model performed:

In [44]:
from sklearn.metrics import classification_report, confusion_matrix

In [45]:
print(confusion_matrix(y_test,predictions,labels=[0,1]))

[[33303   271]
 [ 2140   367]]


In [46]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     33574
           1       0.58      0.15      0.23      2507

    accuracy                           0.93     36081
   macro avg       0.76      0.57      0.60     36081
weighted avg       0.91      0.93      0.91     36081



Not bad! Looks like we only miss-classified one bottle of wine in our test data! This is pretty good considering how few lines of code we had to write. The downside however to using a Multi-Layer Preceptron model is how difficult it is to interpret the model itself. The weights and biases won't be easily interpretable in relation to which features are important to the model itself.

However, if you do want to extract the MLP weights and biases after training your model, you use its public attributes **coefs_** and **intercepts_**. 

**coefs_** is a list of weight matrices, where weight matrix at index i represents the weights between layer i and layer i+1. 

**intercepts_** is a list of bias vectors, where the vector at index i represents the bias values added to layer i+1.

In [47]:
len(mlp.coefs_)

3

In [48]:
len(mlp.coefs_[0])

10

In [49]:
len(mlp.intercepts_[0])

100

## Pipeline

In [50]:
from sklearn.pipeline import Pipeline

# scaler = StandardScaler()
# mlp = MLPClassifier()

estimators = [('scaler', scaler), ('mlp', mlp)]

pipe = Pipeline(estimators)
pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(early_stopping=True,
                               hidden_layer_sizes=(100, 100), max_iter=500,
                               random_state=101))])

In [51]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(early_stopping=True,
                               hidden_layer_sizes=(100, 100), max_iter=500,
                               random_state=101))])

In [52]:
predictions = mlp.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     33574
           1       0.58      0.15      0.23      2507

    accuracy                           0.93     36081
   macro avg       0.76      0.57      0.60     36081
weighted avg       0.91      0.93      0.91     36081

